<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/EfficientNet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Datasets 받기
!curl -L "https://public.roboflow.com/ds/MezJVvNxnf?key=KQhuJSVbjH" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1237      0 --:--:-- --:--:-- --:--:--  1235
100 5082k  100 5082k    0     0  3517k      0  0:00:01  0:00:01 --:--:--  174M
Archive:  roboflow.zip
 extracting: test/images/BloodImage_00334_jpg.rf.3b8a84d57940aeb45e5c2046c8411996.jpg  
 extracting: test/images/BloodImage_00369_jpg.rf.99ae139e5530a25980b7acdd56a4317c.jpg  
 extracting: test/images/BloodImage_00266_jpg.rf.6d62684a33e2f5bc048803aba3177f58.jpg  
 extracting: test/images/BloodImage_00120_jpg.rf.6742a4da047e1226a181d2de2978ce6d.jpg  
 extracting: test/images/BloodImage_00113_jpg.rf.a6d6a75c0ebfc703ecff95e2938be34d.jpg  
 extracting: test/images/BloodImage_00275_jpg.rf.9108b9a016fadd9c367b05dfb0c40c2c.jpg  
 extracting: test/images/BloodImage_00350_jpg.rf.1a19e9f9f197fbeab278718f7c6cea9b.jpg  
 extracting: test/images/BloodImage_00227_jpg

In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=ce80fb0879dad7f918dcbfcfbcd137b323f58e1d59b2c146578c2956115e1094
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [16]:
!pip install torchvision

Load an EfficientNet

In [2]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b0')

Weights까지 불러오기

In [3]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

print(img.shape)

features = model.extract_features

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


Datasets 훈련

In [21]:
# 모듈 필요한 것 여기다가 적기

import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from glob import glob
import yaml

In [8]:
train_img_list = glob('/content/Cell/train/images/*.jpg')
test_img_list = glob('/content/Cell/test/images/*.jpg')
val_img_list = glob('/content/Cell/valid/images/*.jpg')

print(len(train_img_list))
print(len(test_img_list))
print(len(val_img_list))

255
36
73


In [20]:
with open('/content/Cell/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/Cell/valid.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

with open('/content/Cell/test.txt', 'w') as f:
    f.write('\n'.join(test_img_list) + '\n')

In [22]:
with open('/content/Cell/data.yaml', 'r') as f:
    data = yaml.load(f) # json 부르는 거랑 똑같이

print(data)

data['train'] = '/content/Cell/train.txt'
data['val'] = '/content/Cell/val.txt'

with open('/content/Cell/data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['Platelets', 'RBC', 'WBC']}
{'train': '/content/Cell/train.txt', 'val': '/content/Cell/val.txt', 'nc': 3, 'names': ['Platelets', 'RBC', 'WBC']}


In [31]:
batch_size  = 128
random_seed = 555
random.seed(random_seed)
torch.manual_seed(random_seed)

from torchvision import transforms, datasets
from torch.utils.data import Subset

datasets = {}
datasets['train'] = Subset(president_dataset, train_idx)
tmp_dataset       = Subset(president_dataset, tmp_idx)

val_idx, test_idx = train_test_split(list(range(len(tmp_dataset))), test_size=0.5, random_state=random_seed)
datasets['valid'] = Subset(tmp_dataset, val_idx)
datasets['test']  = Subset(tmp_dataset, test_idx)

## data loader 선언
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=4)
dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=4)
dataloaders['test']  = torch.utils.data.DataLoader(datasets['test'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=4)
batch_num['train'], batch_num['valid'], batch_num['test'] = len(dataloaders['train']), len(dataloaders['valid']), len(dataloaders['test'])
print('batch_size : %d,  tvt : %d / %d / %d' % (batch_size, batch_num['train'], batch_num['valid'], batch_num['test']))

TypeError: ignored

In [29]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('----------')

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss, running_corrects, num_cnt = 0.0, 0, 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterioin(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f}'.format(phase, epoch_loss, epoch_acc))
           
            if phase == 'valid' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'Cellmodels.pt')
    print('model saved')

    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model.parameters(), 
                         lr = 0.05,
                         momentum=0.9,
                         weight_decay=1e-4)

lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda)

In [30]:
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=300)

Epoch 0/299
----------


AttributeError: ignored